# Support Vector Machine Learning Model (93% variance - F1 Macro)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.model_selection import train_test_split # for splitting the dataset into training and testing sets
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (93% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.93)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.000982,0.010623,-0.000515,0.007994,-0.015598,0.006235,-0.001005,0.000906,-0.004443,0.000915
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,-0.001904,-0.005919,-0.000364,0.003455,0.001257,0.002125,0.000415,0.000730,-0.001657,-0.002824
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.001164,-0.001459,-0.004173,-0.002534,-0.002817,-0.008647,-0.001667,0.002286,-0.001806,0.004022
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.010939,0.007403,-0.013309,-0.009802,0.004645,0.006029,0.002966,0.008718,0.002546,0.012034
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.013447,0.003492,-0.004594,-0.007241,0.003567,0.005179,-0.002180,-0.001580,-0.002963,0.000785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.004389,0.005525,0.000439,-0.003265,0.005659,0.002306,-0.001173,0.001325,-0.002721,-0.012256
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.001374,0.001591,-0.009597,-0.006642,-0.012765,-0.007957,-0.002451,-0.003381,0.000409,0.002051
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.006847,-0.002853,-0.009210,-0.013922,-0.000218,0.013677,0.012876,0.006643,-0.001759,-0.003755
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.000894,0.000916,-0.002209,-0.001359,-0.000444,0.001107,-0.003264,-0.003120,0.000809,-0.000763


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.000982,0.010623,-0.000515,0.007994,-0.015598,0.006235,-0.001005,0.000906,-0.004443,0.000915
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,-0.001904,-0.005919,-0.000364,0.003455,0.001257,0.002125,0.000415,0.000730,-0.001657,-0.002824
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.001164,-0.001459,-0.004173,-0.002534,-0.002817,-0.008647,-0.001667,0.002286,-0.001806,0.004022
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.010939,0.007403,-0.013309,-0.009802,0.004645,0.006029,0.002966,0.008718,0.002546,0.012034
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.013447,0.003492,-0.004594,-0.007241,0.003567,0.005179,-0.002180,-0.001580,-0.002963,0.000785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.004389,0.005525,0.000439,-0.003265,0.005659,0.002306,-0.001173,0.001325,-0.002721,-0.012256
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.001374,0.001591,-0.009597,-0.006642,-0.012765,-0.007957,-0.002451,-0.003381,0.000409,0.002051
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.006847,-0.002853,-0.009210,-0.013922,-0.000218,0.013677,0.012876,0.006643,-0.001759,-0.003755
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.000894,0.000916,-0.002209,-0.001359,-0.000444,0.001107,-0.003264,-0.003120,0.000809,-0.000763


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (93% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# scale the dataset before PCA
test_set_rescaled = scaler.transform(test_set_features)

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,-0.006063,0.006069,0.000993,0.003714,0.002089,-0.003452,0.002745,-0.005402,0.000027,0.003555
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,-0.009118,-0.009725,-0.010545,-0.004851,0.014564,-0.021674,0.018235,-0.000581,0.002019,-0.003036
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,0.003434,-0.004330,-0.000294,0.009968,-0.008825,-0.005980,0.008058,0.001360,0.002622,-0.004397
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,0.007854,0.003238,-0.004436,0.000426,0.003361,0.001297,-0.000838,0.002522,0.002150,0.003649
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,0.001036,-0.000768,0.003514,0.000141,0.002668,-0.000743,0.001695,0.000717,0.002058,0.003314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,-0.000349,0.006163,0.001671,-0.002955,0.003619,-0.000013,-0.003283,0.001934,0.008094,-0.000615
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,-0.010884,0.002863,0.008247,-0.002358,-0.010809,0.012228,0.000217,-0.003049,-0.006764,-0.012679
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,-0.005772,-0.008498,0.016681,-0.001892,-0.006727,-0.003911,-0.009617,0.002039,0.002130,-0.010287
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,-0.010926,0.003362,0.002041,-0.007122,0.004304,0.001877,0.000283,-0.003580,-0.002617,-0.003368


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.5min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.5min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.1min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.9min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.8min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.8min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.6min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.6min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.6min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.4min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [10]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_93_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_93_f1macro.csv